In [ ]:
import import_ipynb
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from model import build_unet
from metrics import dice_loss, dice_coef, iou

# Rest of the code...




def create_dir(path):
    """ Create a directory if it doesn't exist. """
    os.makedirs(path, exist_ok=True)


def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "CXR_png", "*.png")))
    masks1 = sorted(glob(os.path.join(path, "GTMask", "leftMask", "*.png")))
    masks2 = sorted(glob(os.path.join(path, "GTMask", "rightMask", "*.png")))

    train_x, valid_x, train_y1, valid_y1, train_y2, valid_y2 = train_test_split(
        images, masks1, masks2, test_size=split, random_state=42
    )

    train_x, test_x, train_y1, test_y1, train_y2, test_y2 = train_test_split(
        train_x, train_y1, train_y2, test_size=split, random_state=42
    )

    return (train_x, train_y1, train_y2), (valid_x, valid_y1, valid_y2), (test_x, test_y1, test_y2)


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x.astype(np.float32) / 255.0
    return x


def read_mask(path1, path2):
    x1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
    x2 = cv2.imread(path2, cv2.IMREAD_GRAYSCALE)
    x = x1 + x2
    x = cv2.resize(x, (W, H))
    x = x.astype(np.float32) / np.max(x)
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x


def tf_parse(x, y1, y2):
    def _parse(x, y1, y2):
        x = x.decode()
        y1 = y1.decode()
        y2 = y2.decode()

        x = read_image(x)
        y = read_mask(y1, y2)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y1, y2], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y


def tf_dataset(X, Y1, Y2, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y1, Y2))
    dataset = dataset.shuffle(buffer_size=200)
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Global parameters """
    H = 512
    W = 512

    """ Hyperparameters """
    batch_size = 2
    lr = 1e-5
    num_epochs = 20
    model_path = os.path.join("files2", "model")
    csv_path = os.path.join("files2", "data.csv")

    """ Dataset """
    dataset_path = "C:\\Users\\IIITSKLM\\Desktop\\indiana"

    (train_x, train_y1, train_y2), (valid_x, valid_y1, valid_y2), (test_x, test_y1, test_y2) = load_data(dataset_path)
    print(f"Train: {len(train_x)} - {len(train_y1)} - {len(train_y2)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y1)} - {len(valid_y2)}")
    print(f"Test: {len(test_x)} - {len(test_y1)} - {len(test_y2)}")

    train_dataset = tf_dataset(train_x, train_y1, train_y2, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y1, valid_y2, batch=batch_size)

    """ Model """
    model = build_unet((H, W, 3))
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

    callbacks = [
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path)
    ]

    print("Training started...")
    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        callbacks=callbacks
    )
    print("Training completed.")

    """ Save the model """
    model.save(model_path, save_format='tf')

    """ Inspect the data """
    sample_idx = 0  # Index of the sample to visualize
    sample_image = read_image(train_x[sample_idx])
    sample_mask = read_mask(train_y1[sample_idx], train_y2[sample_idx])

    # Print the shapes of the sample image and mask
    print("Sample Image Shape:", sample_image.shape)
    print("Sample Mask Shape:", sample_mask.shape)

    # Visualize the sample image and mask
    cv2.imshow("Sample Image", sample_image)
    cv2.imshow("Sample Mask", sample_mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


importing Jupyter notebook from model.ipynb
2.13.0
importing Jupyter notebook from metrics.ipynb
Train: 44 - 44 - 44
Valid: 6 - 6 - 6
Test: 5 - 5 - 5
Training started...
Epoch 1/20
22/22 [==============================] - 711s 32s/step - loss: 0.6296 - dice_coef: 0.3704 - iou: 0.2282 - recall: 0.3165 - precision: 0.4754 - val_loss: 0.6939 - val_dice_coef: 0.3061 - val_iou: 0.1810 - val_recall: 0.1876 - val_precision: 0.4175 - lr: 1.0000e-05
Epoch 2/20
22/22 [==============================] - 714s 32s/step - loss: 0.5204 - dice_coef: 0.4796 - iou: 0.3164 - recall: 0.6102 - precision: 0.5425 - val_loss: 0.6956 - val_dice_coef: 0.3044 - val_iou: 0.1797 - val_recall: 0.0777 - val_precision: 0.2940 - lr: 1.0000e-05
Epoch 3/20
22/22 [==============================] - 745s 34s/step - loss: 0.4303 - dice_coef: 0.5697 - iou: 0.3997 - recall: 0.7518 - precision: 0.6089 - val_loss: 0.7001 - val_dice_coef: 0.2999 - val_iou: 0.1766 - val_recall: 0.0099 - val_precision: 0.0898 - lr: 1.0000e-05
Epoch

INFO:tensorflow:Assets written to: files2\model\assets


Sample Image Shape: (512, 512, 3)
Sample Mask Shape: (512, 512, 1)
